# Getting started with ConcatenatedShorQubit and ShorCircuit

The shor_code_package provides tools for building Qiskit Quantum circuits of logical qubits encoded in concatenations of the Shor code. Below is a set of examples that should get you started using the package.

In [ ]:
#Import required packages
from shor_code_package.shor_code import ShorCircuit, ConcatenatedShorQubit
from qiskit import ClassicalRegister, generate_preset_pass_manager
from qiskit_aer import AerSimulator

# The ConcatenatedShorQubit class

The ConcatenatedShorQubit class provides the gates for use for a qubit encoded using a given $n$. $n=0$ is no encoding, $n=1$ is the Shor code, $n=2$ is the Shor code concatenated with itself etc.

In [ ]:
n = 1
csq = ConcatenatedShorQubit(n)

display("Encoder")
display(csq.encoder().draw(fold=-1))

display("Logical X")
display(csq.logical_X().draw(fold=-1))

display("Logical Z")
display(csq.logical_Z().draw(fold=-1))

display("Logical H")
display(csq.logical_H().draw(fold=-1))

display("Logical S")
display(csq.logical_S().draw(fold=-1))

display("Error correction")
display(csq.syndrome_correction_circuit().decompose().draw(fold=-1))

The circuits provided by the ConcatenatedShorQubit can be used as is, but is more commonly used in the context of a ShorCircuit!

## Creating a ShorCircuit

A Shor circuit is initialized with a list of integers representing the level of encoding of each qubit in the circuit.

In [ ]:
sc = ShorCircuit([0,1])
sc.x(0)
sc.h(1)

#Display Shor circuit
display("Shor circuit")
display(sc.draw(fold=-1))

#Export the circuit. The circuit is decomposed.
qc = sc.get_circuit()
display("Shor circuit decomposed")
display(qc.draw(fold=-1))

Encoders are by default added. Qubit 10 is an ancilla qubit used for syndrome measurements during error correction.

Now let's try inserting error correction, and run a simple experiment.

In [ ]:
sc = ShorCircuit([1])
sc.h(0)
sc.s(0)
sc.error_correct(0)
sc.save_stabilizer(label='s')

#Add a classical register and measure
classical_register = ClassicalRegister(1 + sc.num_classical_bits) #Must also contain enough bits for the stabilizer measurements
sc.add_register(classical_register)
sc.measure(0, classical_register[0])

#Display Shor circuit
display("Shor circuit")
display(sc.draw(fold=-1))

#Export the circuit. The circuit is decomposed.
qc = sc.get_circuit(classical_register=classical_register) #Provide the classical register when exporting.
display("Shor circuit decomposed")
display(qc.draw(fold=-1))

In [ ]:
#Set up simulator. Stabilizer method is useful when n > 1 since statevector simulation is intractable when getting to 81 qubits.
aer = AerSimulator(method='stabilizer')

#Get pass manager
pass_manager = generate_preset_pass_manager(1, aer) #optimization level should be 1 otherwise aer can simulate using stabilizer method.

qc = pass_manager.run(sc.get_circuit(classical_register = classical_register))
display(qc.draw(fold=-1))

result = aer.run(qc, shots=1000).result()

In [ ]:
display(result.get_counts())
display(result.data()['s'])
display(result.data()['s'].probabilities_dict())

### Intermediate error correction

The logical H and S gate also support intermediate error correction.

In [ ]:
sc = ShorCircuit([2])
sc.h(0, intermediate_error_correction=True)
#Transpile false means error correction is not inserted but placeholders are left.
display("Hadamard for n=2")
display(sc.get_circuit(transpile=False).decompose().draw(fold=-1)) 

## Multiple measurement schemes
By setting the num_measurements_in_error_correction parameter when creating a ShorCircuit, multiple measurements can be done before doing a recovery action. The majority between the outcomes will determine the recovering action.

In [ ]:
sc = ShorCircuit([1], num_measurements_in_error_correction=3)
sc.error_correct(0)
#When doing multi measurement schemes, a big enough register must be passed manually when exporting the circuit.
classical_register = ClassicalRegister(3*sc.num_classical_bits)
sc.add_register(classical_register)

qc = sc.get_circuit(classical_register=classical_register)
display(qc.draw(fold=-1))

# Example: Preparation of GHZ state.
Let's create a circuit for preparing an encoded GHZ state.

In [ ]:
#Set up the circuit
sc = ShorCircuit([1,0,2]) #Operations between multiple levels of encoding is supported.
sc.h(0)
sc.cx(0,1)
sc.cx(0,2)

classical_register = ClassicalRegister(3 + sc.num_classical_bits)
sc.add_register(classical_register)

sc.save_stabilizer(label='s')

sc.measure(0, classical_register[0])
sc.measure(1, classical_register[1])
sc.measure(2, classical_register[2])

display("Schematic of circuit")
display(sc.draw(fold=-1))

#Simulate
aer = AerSimulator(method='stabilizer')
pass_manager = generate_preset_pass_manager(1, aer)
qc = pass_manager.run(sc.get_circuit(classical_register=classical_register))

result = aer.run(qc, shots=1000).result()

display("Results:")
display("Measurement:")
display(result.get_counts())
display("Stabilizer measurement:")
display(result.data()['s'].measure([0,1,2])[0])